In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [40]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = 'C:/Users/hojun/Desktop/Jun/Study_Note/Pytorch_Tutorial/Dataset/custom_data/train_data'
test_path = 'C:/Users/hojun/Desktop/Jun/Study_Note/Pytorch_Tutorial/Dataset/custom_data/test_data'
save_path = 'C:/Users/hojun/Desktop/Jun/Study_Note/Pytorch_Tutorial/Weight/custom/model.pth'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [41]:
trans = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.ImageFolder(root = data_path, transform=trans)

In [42]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)  # 이 num_workers는 뭘까

In [43]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
        nn.Conv2d(3,6,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
        nn.Linear(16*13*29, 120),
        nn.ReLU(),
        nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
#         print(out.shape)                    # 이런식으로 out.shape를 하나씩 출력해보면 모델을 구성하기 훨씬 쉽다
        out = self.layer2(out)
#         print(out.shape)
        out = out.view(out.shape[0], -1)
#         print(out.shape)                    # 주의사항은 이런식으로 테스트를 하고나면 꼭 주석처리를 해줘야 한다고 하시네.. 왜그럴까
                                            # 딱 보니까 여기 주석 안되어있으면 콘솔이 개판이 되는구먼
        out = self.layer3(out)
        return out

In [44]:
# testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [45]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
loss_func = nn.CrossEntropyLoss().to(device)

In [46]:
total_batch = len(data_loader)

epochs = 5
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    
    print('[Epoch:{}] cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.6827905178070068
[Epoch:2] cost = 0.6472885012626648
[Epoch:3] cost = 0.5996397137641907
[Epoch:4] cost = 0.5357434153556824
[Epoch:5] cost = 0.4519674777984619
Learning Finished!


In [47]:
torch.save(net.state_dict(), save_path)

In [48]:
new_net = CNN().to(device)  # 테스트를 위한 새로운 네트워크 선언

In [49]:
new_net.load_state_dict(torch.load(save_path))

<All keys matched successfully>

In [50]:
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])       # 이게 뭘 보여주는건지 감이 잘 안오네
print(new_net.layer1[0].weight[0][0][0])

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0964, -0.0019, -0.0222, -0.0262,  0.0881], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.0964, -0.0019, -0.0222, -0.0262,  0.0881], device='cuda:0',
       grad_fn=<SelectBackward>)


In [51]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root=test_path, transform = trans)

In [52]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [53]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = new_net(imgs)
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.9799999594688416
